# Diet Metrics Definition & QA 

### Data Analytics & Data Science

### Objectives

-   Document the most critical Diet metrics
-   Source of truth for both Data Analytics & Data Science teams
-   Run quick QA on these most critical metrics

<h3>Table of Contents</h3>

<div class="alert alert-block alert-info" style="margin-top: 15px">
<ol>
    <li><a href="#acquisition"><b>Acquisition</b></a>
        <ul>
            <li><a href="#primary_sales">Primary Sales</a>
            <li><a href="#upsells">Upsells</a>
            <li><a href="#renewals">Renewals</a>
            <li><a href="#gross_revenue">Gross Revenue</a>   
            <li><a href="#aov">AOV (Average Order Value)</a>
            <li><a href="#adspent">AdSpent</a>
            <li><a href="#cpa">CPA (Cost Per Acquisition)</a>
            <li><a href="#unique_sessions">Unique Sessions</a>
            <li><a href="#cr">CR (Conversion Rate)</a>
        </ul>
<br>
    <li><a href="#retention"><b>Retention</b></a></li>
        <ul>
            <li><a href="#retention_metric_1">Retention Metric 1</a>
            <li><a href="#retention_metric_2">Retention Metric 2</a>
            <li><a href="#retention_metric_3">Retention Metric 3</a>
            <li><a href="#retention_metric_4">Retention Metric 4</a>
            <li><a href="#retention_metric_4">Retention Metric 5</a>
        </ul>
<br>
    <li><a href="#others"><b>Others</b></a></li>
        <ul>
            <li><a href="#kcollagen">Kcollagen Cost</a>
        </ul>
</ol>

</div>
<hr>

<h3>Deal with test users</h3>

<div class="alert alert-warning" style="margin-top: 15px">

Test users are excluded from all the following queries. All users whose emails ends with <b>'@teamcmp.com'</b> and <b>'@moarmarketing.com'</b> are considered as test users, and therefore, excluded from all the following results.

</div>
<hr>

In [19]:
import sys
import os
import yaml
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
from datetime import timedelta
from cmpdata.databases.util import read_rd_query, read_legacy_query, get_sql_query
from utils import functions

import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'utils'

In [2]:
db_config = yaml.safe_load(open(os.path.expanduser('~/.dbconfig/db_config_parameters.yml'), 'r'))

In [3]:
START_DATE = '2021-01-01'

<h2 id="acquisition">Acquisition</h2>
<h3 id="primary_sales">Primary Sales</h3>
<br>
Primary sales are subscriptions to 1,2,3,6 or 12 months plans. 
<br>

<b>Table</b>: <code>kb.membership_created</code>
<br>

- <code>product_id</code> <b>=</b> <code>1</code>

In [5]:
query = get_sql_query(os.path.join(os.getcwd(),'data/sql/primary_sales_query.sql')) % params['start_date']

df_primary_sales = read_rd_query(query, **db_config['redshift_config'])

# Convert to a data frame and sort by date DESC
df_primary_sales = df_primary_sales.sort_values(by='date', ascending=False)

# Show last 7 days Primary Sales
df_primary_sales.head(7)

NameError: name 'get_sql_query' is not defined

In [9]:
df_primary_sales.tail(10)

,date,primary_sales
199,2021-01-10,382
200,2021-01-09,386
201,2021-01-08,289
202,2021-01-07,321
203,2021-01-06,322
204,2021-01-05,290
205,2021-01-04,325
206,2021-01-03,837
207,2021-01-02,836
208,2021-01-01,612


In [12]:
# save to data folder
df_primary_sales.to_csv(
    os.path.join('data/processed', 'daily_primary_{}.csv'.format(pd.to_datetime('now').strftime('%d%m%y'))))

<u><h3 id="upsells">Upsells</h3></u>
<br>
Upsells are additional products users can buy after subscribing to a meal plan (primary sale). 
<br>

<b>Table</b>: <code>kb.membership_created</code>
<br>

- <code>product_id</code> <b>!=</b> <code>1</code>

In [8]:
# Diet - Upsells SQL query
#START_DATE = '2021-06-01 00:00:00'
upsells_query = f"""
SELECT
    DATE(mc.created_at) AS date,
    COUNT(DISTINCT mc.membership_id) AS upsells
FROM
    kb.membership_created mc
LEFT JOIN kb.user_registered ur ON
    ur.user_id = mc.user_id
WHERE
    mc.product_id != 1
    AND mc.created_at >= '{START_DATE}'
    AND ur.email NOT LIKE '%%@teamcmp.com'
    AND ur.email NOT LIKE '%%@moarmarketing.com'
GROUP BY
    DATE(mc.created_at)
ORDER BY 
    DATE(mc.created_at) DESC
"""

In [9]:
# Read query
upsells = read_rd_query(upsells_query, **db_config['redshift_config'])
upsells = upsells.copy(deep=True)

# Convert to a data frame and sort by date DESC
df_upsells = pd.DataFrame(upsells).sort_values(by='date', ascending=False)

# Show last 7 days Upsells
df_upsells.head(7)

,date,upsells
0,2021-07-26,85
1,2021-07-25,47
2,2021-07-24,43
3,2021-07-23,114
4,2021-07-22,30
5,2021-07-21,32
6,2021-07-20,37


In [11]:
# save to data folder
df_upsells.to_csv(
    os.path.join('data/processed', 'daily_upsells_{}.csv'.format(pd.to_datetime('now').strftime('%d%m%y'))))

<u><h3 id="renewals">Renewals</h3></u>
<br>
Renewals are rebilled transactions on the 1,2,3,6 or 12 months plans.
<br>

<b>Table</b>: <code>kb.recurly_transactions</code>


- <code>origin</code> <b>=</b> <code>'recurring'</code>
- <code>status</code> <b>=</b> <code>'success'</code>

In [8]:
# Diet - Renewals SQL query
#START_DATE = '2021-06-01 00:00:00'
renewals_query = f"""
SELECT
    DATE(t."date") AS renewal_date,
    COUNT(DISTINCT t.transaction_id) AS renewals
FROM
    kb.recurly_transactions t
LEFT JOIN kb.recurly_accounts ra ON
    ra.account_code = t.account_code
WHERE
    t.origin = 'recurring'
    AND t.status = 'success'
    AND t."date" >= '{START_DATE}'
    AND ra.account_email NOT LIKE '%%@teamcmp.com'
    AND ra.account_email NOT LIKE '%%@moarmarketing.com'
GROUP BY
    DATE(t."date")
ORDER BY
    DATE(t."date") DESC
"""

In [9]:
# Read query
renewals = read_rd_query(renewals_query, **db_config['redshift_config'])
renewals = renewals.copy(deep=True)

# Convert to a data frame and sort by renewal_date DESC
df_renewals = pd.DataFrame(renewals).sort_values(by='renewal_date', ascending=False)

# Show last 7 days Renewals
df_renewals.head(7)

,renewal_date,renewals
0,2021-07-01,159
1,2021-06-30,195
2,2021-06-29,143
3,2021-06-28,166
4,2021-06-27,139
5,2021-06-26,131
6,2021-06-25,107


<u><h3 id="gross_revenue">Gross Revenue</h3></u>
<br>
The Gross Revenue represents the total amount spent by members in primary plans, upsells and complements.
<br>

As we operate on multiple currencies, the Gross Revenue needs to be converted in USD
<br>

<b>Tables</b>: <code>kb.membership_created</code> & <code>etl.currency_conversions</code>

In [10]:
# Diet - Gross Revenue query
#START_DATE = '2021-06-01 00:00:00'
gross_revenue_query = f"""
SELECT
    DATE(mc.created_at) AS date,
    ROUND(SUM((mc.amount * COALESCE(cc.usd_conversion_rate,1))),0) AS gross_revenue_usd
FROM
    kb.membership_created mc
LEFT JOIN kb.user_registered ur ON
    ur.user_id = mc.user_id
LEFT JOIN etl.currency_conversions cc ON
    (DATE(cc.date_cest) = (DATE(mc.created_at))
    AND (DATE_PART(hour, mc.created_at) = cc.hour_cest)
    AND (cc.currency = mc.currency))
WHERE
    mc.created_at >= '{START_DATE}'
    AND ur.email NOT LIKE '%%@teamcmp.com'
    AND ur.email NOT LIKE '%%@moarmarketing.com'
GROUP BY
    DATE(mc.created_at)
ORDER BY 
    DATE(mc.created_at) DESC
"""

In [11]:
# Read query
gross_revenue = read_rd_query(gross_revenue_query, **db_config['redshift_config'])
gross_revenue = gross_revenue.copy(deep=True)

# Convert to a data frame and sort by date DESC
df_gross_revenue = pd.DataFrame(gross_revenue).sort_values(by='date', ascending=False)

# Show last 7 days Gross Revenue
df_gross_revenue.head(7)

,date,gross_revenue_usd
0,2021-07-02,1435.0
1,2021-07-01,4182.0
2,2021-06-30,3617.0
3,2021-06-29,3222.0
4,2021-06-28,2212.0
5,2021-06-27,2069.0
6,2021-06-26,1755.0


<u><h3 id="aov">AOV (Average Order Value)</h3></u>
<br>
The AOV (Average Order Value) represents the average amount spent beach time a member places an order.
<br>

<b>Table</b>: <code>kb.membership_created</code>


- <code>AOV Primary Sales</code> <b>=</b> <code>Gross Revenue USD from Primary Sales / Primary Sales</code>
- <code>AOV Upsells</code> <b>=</b> <code>Gross Revenue USD from Upsells / Primary Sales </code>
- <code>AOV Total</code> <b>=</b> <code>(Gross Revenue USD from Primary Sales + Upsells) / Primary Sales </code>

In [12]:
# Diet - AOV SQL query
#START_DATE = '2021-06-01 00:00:00'
aov_query = f"""
SELECT
    DATE(mc.created_at) AS date,
    (CASE
        mc.product_id WHEN mc.product_id = 1 THEN 'primary'
        ELSE 'upsells'
        END) AS product_type,
    COUNT(DISTINCT mc.membership_id) AS transactions,
    ROUND(SUM((mc.amount * COALESCE(cc.usd_conversion_rate,1))),0) AS gross_revenue_usd,
    ROUND((ROUND(SUM((mc.amount * COALESCE(cc.usd_conversion_rate,1))),0) / COUNT(DISTINCT mc.membership_id)),2) AS aov
FROM
    kb.membership_created mc
LEFT JOIN kb.user_registered ur ON
    ur.user_id = mc.user_id
LEFT JOIN etl.currency_conversions cc ON
    (DATE(cc.date_cest) = (DATE(mc.created_at))
    AND (DATE_PART(hour, mc.created_at) = cc.hour_cest)
    AND (cc.currency = mc.currency))
WHERE
    mc.created_at >= '{START_DATE}'
    AND ur.email NOT LIKE '%%@teamcmp.com'
    AND ur.email NOT LIKE '%%@moarmarketing.com'
GROUP BY
    DATE(mc.created_at),
    (CASE
        mc.product_id WHEN mc.product_id = 1 THEN 'primary'
        ELSE 'upsells'
        END)
ORDER BY 
    DATE(mc.created_at) DESC,
    product_type ASC
"""

In [13]:
# Total AOV calculation using both df_primary_sales and df_gross_revenue
df_total_aov = df_gross_revenue
total_aov = round((df_gross_revenue['gross_revenue_usd']/primary_sales['primary_sales']),2)
df_gross_revenue['total_aov'] = total_aov

# Show last 7 days Total AOV
df_gross_revenue.head(7)

,date,gross_revenue_usd,total_aov
0,2021-07-02,1435.0,51.25
1,2021-07-01,4182.0,50.39
2,2021-06-30,3617.0,49.55
3,2021-06-29,3222.0,51.14
4,2021-06-28,2212.0,50.27
5,2021-06-27,2069.0,49.26
6,2021-06-26,1755.0,48.75


In [14]:
# Read query
aov = read_rd_query(aov_query, **db_config['redshift_config'])
aov = aov.copy(deep=True)

# Convert to a data frame and sort by date DESC
df_aov = pd.DataFrame(aov).sort_values(by='date', ascending=False)

# Show last 7 days AOV broken down per Primary Sales and Upsells
df_aov.head(14)

,date,product_type,transactions,gross_revenue_usd,aov
0,2021-07-02,primary,28,1332.0,47.57
1,2021-07-02,upsells,16,103.0,6.44
2,2021-07-01,primary,83,3954.0,47.64
3,2021-07-01,upsells,45,228.0,5.07
4,2021-06-30,primary,73,3435.0,47.05
5,2021-06-30,upsells,32,182.0,5.69
6,2021-06-29,primary,63,2936.0,46.60
7,2021-06-29,upsells,50,286.0,5.72
8,2021-06-28,primary,44,2067.0,46.98
9,2021-06-28,upsells,27,145.0,5.37


<h3 id="adspent">AdSpent</h3>
<br>
Budget spent across all Ad Marketing plateforms (Facebook Ads, Google Ads etc.)
<br>

<b>Table</b>: <code>funnel_io.adspent</code>
<br>

- <code>product_name</code> <b>=</b> <code>KetoBody</code>

In [15]:
# Diet - AdSpent Query
#START_DATE = '2021-06-01 00:00:00'
adspent_query = f"""
SELECT
    DATE(a.date) AS date,
    ROUND(SUM(a.adspent),2) AS adspent
FROM
    funnel_io.adspent a
WHERE
    a.product_name = 'KetoBody'
    AND a.date >= '{START_DATE}'
GROUP BY
    DATE(a.date)
ORDER BY
    DATE(a.date) DESC
"""

In [16]:
# Read query
adspent = read_rd_query(adspent_query, **db_config['redshift_config'])
adspent = adspent.copy(deep=True)

# Convert to a data frame and sort by date DESC
df_adspent = pd.DataFrame(adspent).sort_values(by='date', ascending=False)

# Show last 7 days AdSpent
df_adspent.head(7)

,date,adspent
0,2021-07-02,968.38
1,2021-07-01,5905.24
2,2021-06-30,5642.38
3,2021-06-29,4269.71
4,2021-06-28,2637.19
5,2021-06-27,2537.58
6,2021-06-26,2534.75


<h3 id="cpa">CPA (Cost Per Acquisition)</h3>
<br>
The CPA (Cost Per Acquisition) measures the aggregate cost to acquire one paying customer.
<br>

<b>Tables</b>: <code>kb.membership_created</code> & <code>funnel_io.adspent</code>
<br>

- <code>CPA</code> <b>=</b> <code>AdSpent / Primary Sales</code>

In [17]:
# CPA calculation using both df_primary_sales and df_adspent
df_cpa = df_adspent
cpa = round((df_adspent['adspent']/df_primary_sales['primary_sales']),2)
df_adspent['cpa'] = cpa

# Show last 7 days CPA
df_cpa.head(7)

,date,adspent,cpa
0,2021-07-02,968.38,34.58
1,2021-07-01,5905.24,71.15
2,2021-06-30,5642.38,77.29
3,2021-06-29,4269.71,67.77
4,2021-06-28,2637.19,59.94
5,2021-06-27,2537.58,60.42
6,2021-06-26,2534.75,70.41


<h3 id="unique_sessions">Unique Sessions</h3>
<br>
Unique sessions to each of our active tours. 
<br>

<b>Table</b>: <code>kb.tour_steps</code>
<br>

- <code>tour_step</code> <b>=</b> <code>0</code>, to catch all visitors starting at the first step of the tour

In [18]:
# Diet - Unique Sessions query
#START_DATE = '2021-06-01 00:00:00'
unique_sessions_query = f"""
SELECT
    DATE(ts.submitted_at) AS date,
    COUNT(DISTINCT ts.session_id) AS unique_sessions
FROM
    kb.tour_steps ts
WHERE
    ts.tour_step = 0
    AND ts.submitted_at >= '{START_DATE}'
GROUP BY
    DATE(ts.submitted_at)
ORDER BY
    DATE(ts.submitted_at) DESC
"""

In [19]:
# Read query
unique_sessions = read_rd_query(unique_sessions_query, **db_config['redshift_config'])
unique_sessions = unique_sessions.copy(deep=True)

# Convert to a data frame and sort by date DESC
df_unique_sessions = pd.DataFrame(unique_sessions).sort_values(by='date', ascending=False)

# Show last 7 days AdSpent
df_unique_sessions.head(7)

,date,unique_sessions
0,2021-07-02,1920
1,2021-07-01,4618
2,2021-06-30,4492
3,2021-06-29,3089
4,2021-06-28,2515
5,2021-06-27,2514
6,2021-06-26,2310


<h3 id="cr">CR (Conversion Rate)</h3>
<br>
The CR (Conversion Rate) measures the conversion between all the users reaching our active tours and the ones who end up subscribing to the product (Primary Sales).
<br>

<b>Tables</b>: <code>kb.membership_created</code> & <code>kb.tour_steps</code>
<br>

- <code>CR</code> <b>=</b> <code>Primary Sales / Unique Sessions</code>

In [20]:
# CR calculation using both df_primary_sales and df_unique_sessions
df_cr = df_unique_sessions
cr = round(((df_primary_sales['primary_sales']/df_unique_sessions['unique_sessions'])*100),2)
df_unique_sessions['cr'] = cr

# Show last 7 days CPA
df_cr.head(7)

,date,unique_sessions,cr
0,2021-07-02,1920,1.46
1,2021-07-01,4618,1.80
2,2021-06-30,4492,1.63
3,2021-06-29,3089,2.04
4,2021-06-28,2515,1.75
5,2021-06-27,2514,1.67
6,2021-06-26,2310,1.56


<h3 id="al">Activity Level</h3>
<br>
Activity Level measures the number of days with at least one login in four windows; 2 days, 7 days, 15 days and 30 days.
<br>

<b>Tables</b>: <code>kb.user_logged</code>
<br>

In [133]:
#START_DATE = '2020-10-01'

logins_query = f"""
SELECT ul.*, rs.email 
FROM kb.user_logged AS ul
    JOIN kb.user_registered AS ur ON (ul.user_id = ur.user_id)
    JOIN kb.recurly_subscriptions AS rs ON (ur.email=rs.email) 
WHERE ul.logged_at >= {START_DATE};
"""

In [143]:
df = read_rd_query(logins_query, **db_config['redshift_config'])
df.head()

,event_id,user_id,membership_id,membership_status,logged_at,email
0,c0e27324-471e-423a-ae48-64ff30f04eec,9418,822,active,2021-04-18 17:07:33,llballam.lb@gmail.com
1,8635474f-02ae-4eac-b4cb-0bbd1d2d670e,9638,77909,active,2021-03-07 15:31:14,mayshallal1@gmail.com
2,667c9f04-3b58-440c-ac1d-195dadd4a573,9638,77909,active,2021-03-08 13:37:16,mayshallal1@gmail.com
3,1a232878-9dda-4b16-b892-f4d65cf803b9,10155,92110,active,2021-04-05 19:55:27,misionmuchacha@gmail.com
4,b4e56fe4-61df-4444-95c3-118ede839ef9,10155,92110,active,2021-04-05 20:06:26,misionmuchacha@gmail.com


In [144]:
# get first logged timestamp
df['first_logged_at'] = df.groupby(['email'])['logged_at'].transform('min')

#### Activity Level Function

In [152]:
def activity_level(df, window):
    df[['first_log_date', 'logged_date']] = df[['first_logged_at', 'logged_at']].values.astype('datetime64[D]')
    df.drop_duplicates(subset=['email', 'logged_date'], keep='first', inplace=True)
    
    time_window = (df['first_log_date'] + timedelta(days=window))
    members = df['membership_id'].nunique()
    act_level = df[df['logged_date']<time_window]['logged_date'].count()
    return act_level/members

In [153]:
print(activity_level(df, 2))
print(activity_level(df, 7))

1.2702056529227028
1.8664978219025428


<h2 id="others">Others</h2>
<h3 id="kcollagen">Kcollagen Cost</h3>
<br>
Shipping costs for Kcollagen complements sold to members.
<br>

<b>Table</b>: <code>kb.membership_created</code>
<br>

- <code>product_id</code> <b>IN</b> <code>(8, 9, 10)</code>

In [20]:
# Diet - Kcollagen cost query
START_DATE = '2021-04-01 00:00:00'
kcollagen_cost_query = f"""
SELECT
    DATE(mc.created_at) AS date,
    mc.product_id,
    p.name AS product_name,
    (CASE
        mc.product_id WHEN mc.product_id = 8 THEN 12.05
        WHEN mc.product_id = 9 THEN 29.90
        WHEN mc.product_id = 10 THEN 44.50
    END) AS cost_per_unit,
    COUNT(DISTINCT mc.membership_id) AS kcollagen_units_sold
FROM
    kb.membership_created mc
LEFT JOIN kb.user_registered ur ON
    ur.user_id = mc.user_id
LEFT JOIN kb.products p ON
    p.id = mc.product_id
WHERE
    mc.product_id IN (8, 9, 10)
    AND mc.created_at >= '2021-04-01 00:00:00'
    AND ur.email NOT LIKE '%%@teamcmp.com'
    AND ur.email NOT LIKE '%%@moarmarketing.com'
GROUP BY
    DATE(mc.created_at),
    mc.product_id,
    p.name
ORDER BY
    DATE(mc.created_at) DESC,
    mc.product_id ASC
"""

In [21]:
# Read query
kcollagen_cost = read_rd_query(kcollagen_cost_query, **db_config['redshift_config'])
kcollagen_cost = kcollagen_cost.copy(deep=True)

# Convert to a data frame and sort by date DESC
df_kcollagen_cost = pd.DataFrame(kcollagen_cost).sort_values(by='date', ascending=False)

# Show last 10 entries
kcollagen_cost.head(10)

,date,product_id,product_name,cost_per_unit,kcollagen_units_sold
0,2021-05-21,9,KCollagen 3-Pack,29.90,1
1,2021-04-20,8,KCollagen 1-Pack,12.05,1
2,2021-04-20,9,KCollagen 3-Pack,29.90,4
3,2021-04-19,8,KCollagen 1-Pack,12.05,8
4,2021-04-19,9,KCollagen 3-Pack,29.90,14
5,2021-04-18,8,KCollagen 1-Pack,12.05,9
6,2021-04-18,9,KCollagen 3-Pack,29.90,11
7,2021-04-18,10,KCollagen 6-Pack,44.50,1
8,2021-04-17,8,KCollagen 1-Pack,12.05,6
9,2021-04-17,9,KCollagen 3-Pack,29.90,9
